In [1]:
!pip install biopython networkx matplotlib

In [2]:
from Bio import pairwise2
from collections import defaultdict
from Bio.pairwise2 import format_alignment

c:\Users\piyus\anaconda3\lib\site-packages\Bio\pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [13]:
def de_bruijn_graph(reads, k):
    edges = []
    nodes = set()

    for read in reads:
        for i in range(len(read) - k + 1):
            edges.append((read[i : i + k - 1], read[i + 1: i + k]))
            nodes.add(read[i : i + k - 1])
            nodes.add(read[i + 1 : i + k])

    graph = {node: [] for node in nodes}
    for edge in edges:
        graph[edge[0]].append(edge[1])

    return graph

In [14]:
def read_fasta(file_path):
    with open(file_path, 'r') as file:
        reads = [line.strip() for line in file if not line.startswith('>')]
    return reads

k = 10
fasta_file_path = 'read.fasta'
reads = read_fasta(fasta_file_path)
graph = de_bruijn_graph(reads, k)
print(graph)

{'agccccttg': ['gccccttgt', 'gccccttgt', 'gccccttgt', 'gccccttgt', 'gccccttgt'], 'agcgagcgg': ['gcgagcggt', 'gcgagcggt', 'gcgagcggt', 'gcgagcggt', 'gcgagcggt'], 'atctccctg': ['tctccctgg', 'tctccctgg', 'tctccctgg', 'tctccctgg', 'tctccctgg'], 'gagcccctt': ['agccccttg', 'agccccttg', 'agccccttg', 'agccccttg', 'agccccttg'], 'cacccctga': ['acccctgaa', 'acccctgaa', 'acccctgaa', 'acccctgaa', 'acccctgaa', 'acccctgaa', 'acccctgaa'], 'acgcgatga': ['cgcgatgaa', 'cgcgatgaa', 'cgcgatgaa', 'cgcgatgaa', 'cgcgatgaa'], 'cgtgggtgg': ['gtgggtggt', 'gtgggtggt', 'gtgggtggt', 'gtgggtggt', 'gtgggtggt'], 'cgagcggta': ['gagcggtag', 'gagcggtag', 'gagcggtag', 'gagcggtag', 'gagcggtag'], 'tactgtacc': ['actgtacca', 'actgtacca', 'actgtacca', 'actgtacca', 'actgtacca'], 'atcatatga': ['tcatatgat', 'tcatatgat', 'tcatatgat', 'tcatatgat', 'tcatatgat'], 'ccaatctac': ['caatctacc', 'caatctacc', 'caatctacc', 'caatctacc', 'caatctacc', 'caatctacc'], 'ttttcacag': ['tttcacagt', 'tttcacagt', 'tttcacagt', 'tttcacagt', 'tttcacagt'], 

In [15]:
def calculate_degrees(graph):
    degrees = {node: {"in": 0, "out": len(graph[node])} for node in graph}

    for node in graph:
        for neighbor in graph[node]:
            degrees[neighbor]["in"] += 1
    return degrees

degrees = calculate_degrees(graph)
print(degrees)

{'agccccttg': {'in': 5, 'out': 5}, 'agcgagcgg': {'in': 5, 'out': 5}, 'atctccctg': {'in': 5, 'out': 5}, 'gagcccctt': {'in': 5, 'out': 5}, 'cacccctga': {'in': 8, 'out': 7}, 'acgcgatga': {'in': 5, 'out': 5}, 'cgtgggtgg': {'in': 5, 'out': 5}, 'cgagcggta': {'in': 5, 'out': 5}, 'tactgtacc': {'in': 5, 'out': 5}, 'atcatatga': {'in': 5, 'out': 5}, 'ccaatctac': {'in': 6, 'out': 6}, 'ttttcacag': {'in': 5, 'out': 5}, 'ggttctctc': {'in': 5, 'out': 5}, 'agaaactga': {'in': 5, 'out': 5}, 'cacagacac': {'in': 5, 'out': 5}, 'tgaagtcct': {'in': 9, 'out': 7}, 'aacccaatc': {'in': 5, 'out': 5}, 'gagcgaagc': {'in': 5, 'out': 5}, 'atgcctagc': {'in': 5, 'out': 5}, 'gttgtccct': {'in': 6, 'out': 7}, 'ttgacgcag': {'in': 5, 'out': 5}, 'gaggtgaca': {'in': 5, 'out': 5}, 'tggccgcaa': {'in': 5, 'out': 5}, 'cattcccac': {'in': 6, 'out': 6}, 'tccctggtg': {'in': 5, 'out': 5}, 'tatcgttcg': {'in': 8, 'out': 8}, 'accgcggtt': {'in': 5, 'out': 5}, 'cggcacccc': {'in': 5, 'out': 7}, 'agacccttt': {'in': 5, 'out': 5}, 'attcccaat': 

In [16]:
start_node = None
for node in degrees:
  if degrees[node]['in'] <= degrees[node]['out']:
    start_node = node
    break

print(start_node)

agccccttg


In [17]:
stack = [start_node]
path = []

while stack:
    current_node = stack[-1]
    if graph[current_node]:
        stack.append(graph[current_node].pop())
    else:
        path.append(stack.pop())

eulerian_path = path[::-1]

In [18]:
print("->".join(eulerian_path))

agccccttg->gccccttgt->ccccttgtc->cccttgtca->ccttgtcat->cttgtcatc->ttgtcatcc->tgtcatcct->gtcatcctc->tcatcctcg->catcctcga->atcctcgat->tcctcgatg->cctcgatga->ctcgatgac->tcgatgact->cgatgacta->gatgactac->atgactaca->tgactacag->gactacagg->actacaggt->ctacaggtc->tacaggtcc->acaggtccg->caggtccga->aggtccgaa->ggtccgaat->gtccgaatg->tccgaatgg->ccgaatggg->cgaatgggc->gaatgggcc->aatgggccc->atgggcccc->tgggccccc->gggccccct->ggccccctg->gccccctga->ccccctgaa->cccctgaac->ccctgaaca->cctgaacaa->ctgaacaac->tgaacaacc->gaacaaccc->aacaaccca->acaacccac->caacccacc->aacccaccc->acccacccc->cccaccccc->ccaccccca->cacccccag->acccccaga->cccccagag->ccccagaga->cccagagat->ccagagatg->cagagatga->agagatgat->gagatgata->agatgatat->gatgatatc->atgatatca->tgatatcag->gatatcaga->atatcagag->tatcagaga->atcagagag->tcagagagg->cagagaggc->agagaggcg->gagaggcga->agaggcgag->gaggcgagt->aggcgagtg->ggcgagtgg->gcgagtggt->cgagtggtt->gagtggttg->agtggttgt->gtggttgtc->tggttgtcc->ggttgtccc->gttgtccct->ttgtccctg->tgtccctgg->gtccctggc->tccctggcc->ccctggcca-

In [19]:
superstring = eulerian_path[0]

for node in eulerian_path[1:]:
  superstring += node[-1]

print(superstring)

agccccttgtcatcctcgatgactacaggtccgaatgggccccctgaacaacccacccccagagatgatatcagagaggcgagtggttgtccctggccatgtcagtggacttgacgcagccattcccactgaacccaatctaccagaacacagacactgtgactaccgtgtgggtcaactggagagaggtgacaacggcacccctgaacaacccacccccagagatgatatagtgatgaagtccttcaaccagatgaagaaactgattcccaatgcgcattgggagattgcgaagggttcacccaagcagaaccgagactactgtaccaaggaggacacccgagttcccgacacaagaccctttgaagaaggtgagtgtccacatcaaggcacacgcttcgatctgagacagtgtgctgacgtactggccaatggtggttctctccttgatctccctggtgagatggtcatcaggtaccaccgcggttttagcgcgtatcgttcgctgtttcataagcgacccctgaacaacccacccccagagatgatatcagagaggcgagtggttgtccctggccatgtcagtggacttgacgcagccattcccactgaacccaatctaccagaacacagacactgtgactaccgtgtgggtcaactggagagaggtgacaacggcacccctgaacaacccacccccagagatgatatacggccctacaggcagtggcaagtctcgattcgcatcaactctacgcgatgaatgttactggaaaccccctggcaaatggtttgatggttacgaccaggagccccttgtcatcctcgatgactacaggtccgaatgggccccctgaacaacccacccccagagatgatatagtgatgaagtccttcaaccagatgaagaaactgattcccaatgcgcattgggagattgcgaagggttcacccaagcagaaccgagactactgtaccaaggaggacacccgagttcccg

In [20]:
key = 'acctcggcaacatccgatcatcatatgatcaattatgactcatcccgcgtggaattatgtagccaatgaaatcgctccatatttcaaaaattgagttttcacagtggccgcaatctataaagccgcgagcgaagcgagcggtaggcattttcagtttgaccaaaatgcctagcaacgcaacaaccaaccgagcccgtgggtggtgcttcaccctgaacaacccacccccagagatgatatcagagaggcgagtggttgtccctggccatgtcagtggacttgacgcagccattcccactgaacccaatctaccagaacacagacactgtgactaccgtgtgggtcaactggagagaggtgacaacggcacccctgaacaacccacccccagagatgatatagtgatgaagtccttcaaccagatgaagaaactgattcccaatgcgcattgggagattgcgaagggttcacccaagcagaaccgagactactgtaccaaggaggacacccgagttcccgacacaagaccctttgaagaaggtgagtgtccacatcaaggcacacgcttcgatctgagacagtgtgctgacgtactggccaatggtggttctctccttgatctccctggtgagatggtcatcaggtaccaccgcggttttagcgcgtatcgttcgctgtttcataagcgacccctgaacaacccacccccagagatgatatacggccctacaggcagtggcaagtctcgattcgcatcaactctacgcgatgaatgttactggaaaccccctggcaaatggtttgatggttacgaccaggagccccttgtcatcctcgatgactacaggtccgaatgggccccctgaacaacccacccccagagatgatatctccttgacc'
alignments = pairwise2.align.globalxx(superstring, key, one_alignment_only=True)
best_alignment = alignments[0]

print("Superstring:")
print(superstring)
print("\nKey:")
print(key)
print("\nAlignment:")
print(format_alignment(*best_alignment))

# Extract and print the global alignment score
alignment_score = best_alignment[2]
print(f"\nGlobal Alignment Score: {alignment_score}")

Superstring:
agccccttgtcatcctcgatgactacaggtccgaatgggccccctgaacaacccacccccagagatgatatcagagaggcgagtggttgtccctggccatgtcagtggacttgacgcagccattcccactgaacccaatctaccagaacacagacactgtgactaccgtgtgggtcaactggagagaggtgacaacggcacccctgaacaacccacccccagagatgatatagtgatgaagtccttcaaccagatgaagaaactgattcccaatgcgcattgggagattgcgaagggttcacccaagcagaaccgagactactgtaccaaggaggacacccgagttcccgacacaagaccctttgaagaaggtgagtgtccacatcaaggcacacgcttcgatctgagacagtgtgctgacgtactggccaatggtggttctctccttgatctccctggtgagatggtcatcaggtaccaccgcggttttagcgcgtatcgttcgctgtttcataagcgacccctgaacaacccacccccagagatgatatcagagaggcgagtggttgtccctggccatgtcagtggacttgacgcagccattcccactgaacccaatctaccagaacacagacactgtgactaccgtgtgggtcaactggagagaggtgacaacggcacccctgaacaacccacccccagagatgatatacggccctacaggcagtggcaagtctcgattcgcatcaactctacgcgatgaatgttactggaaaccccctggcaaatggtttgatggttacgaccaggagccccttgtcatcctcgatgactacaggtccgaatgggccccctgaacaacccacccccagagatgatatagtgatgaagtccttcaaccagatgaagaaactgattcccaatgcgcattgggagattgcgaagggttcacccaagcagaaccgagactactgtaccaaggaggac

In [21]:
def calculate_accuracy(true_sequence, aligned_sequence):
    matches = 0
    total = len(true_sequence)

    for true_char, aligned_char in zip(true_sequence, aligned_sequence):
        if true_char == aligned_char:
            matches += 1

    accuracy = (matches / total) * 100
    return accuracy

In [22]:
alignments = pairwise2.align.globalxx(key, superstring, one_alignment_only=True)
best_alignment = alignments[0]
aligned_true_sequence = best_alignment.seqA
aligned_predicted_sequence = best_alignment.seqB

# Print the alignment
print(format_alignment(*best_alignment))

# Calculate accuracy
accuracy = calculate_accuracy(aligned_true_sequence, aligned_predicted_sequence)
print(f"Accuracy: {accuracy:.2f}%")

a-cc--t---c------g--g-c-a-a---c--a-t---cc----ga------------------t-----ca----------t------c-------at---a-tg-a-t---c--a---att---a-tga-c----tc-a---------------t----c--ccg-------c----g--------tg-----g--a------a------------------t--tat-gt-a-g----cc---aa-----tgaa-a---t----c-----gc---t----------c--------ca------------------ta-t-t-------------------t-c---a-a-aa-a---tt-ga-g----t---t-t-----tca---ca---g-t--g----g---c------c-g-c--a------a-t-------ct-------at----------a-a-----a---g---cc---gcg-----agcg---a--------------a---gcga-----g--c------------g-g-t-a-----g-g---c-a-t--tt-tc-------a-gt---t----t-gac-ca---a-----a---a------t------g--c-c------t---a-----g-------caac-g-----------caac---a------a-c--c--a-----a-----------c--c------g--ag---c----c-cg-t--g------------g-g-tg---gt----g----c----t------t-------------c-acc-----c---t-g--a------a---c-a-a---cc-------c------a-c--ccc-c----agagatgatat-----------c----a----ga-ga-g-----g---c-----g---a--g-----t-g-g-----tt-------g------------t-c----cc----------------t----g